# library import

In [161]:
import numpy as np
import matplotlib.image as img
import matplotlib.pyplot as plt
import matplotlib.colors as colors

# data loading

In [187]:
filename_data = './satisfaction_data.csv'
#data = np.genfromtxt(filename_data, encoding = 'ascii', names=('age', 'workclass', 'fnlwgt', 'educational_num', 'marital_status', 'hours_per_week', 'satisfaction'), delimiter =',', dtype = None)
data_feature = np.loadtxt(filename_data, delimiter =',',usecols= (0,1,2,3,4,5), dtype=int)
data_class = np.loadtxt(filename_data, delimiter =',',usecols= (6), dtype='str')
number_data = data_feature.shape[0]

#genfromtext를 전 영역에 대해서 쓰니까 string 값들이 nan으로 나온다..
#여기서 nan이란 not a number 라는 뜻.
#dtype을 float64로 가져와서 발생하는 현상.
#https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=radii26omg&logNo=221051465120

#print('number of data = ', number_data)
#print(data_feature.shape)
#print(data_feature[0].dtype)
#print(data_class.shape)


int32
[['40' '2' '1' ... '25' '9' '0']
 ['40' '2' '1' ... '54' '10' '1']
 ['40' '2' '1' ... '43' '14' '1']
 ...
 ['40' '2' '1' ... '32' '5' '0']
 ['40' '3' '1' ... '47' '10' '1']
 ['50' '4' '1' ... '35' '13' '1']]


# data inspection

In [188]:
minmax = np.empty((6,2), float)

for i in range(6):
    print("data_col_"+str(i))
    minmax[i][0] = np.min(data_feature, axis=0)[i]
    print("min:"+str(minmax[i][0]))
    minmax[i][1] = np.max(data_feature, axis=0)[i]
    print("max:"+str(minmax[i][1]))
    print("mean:"+str(np.mean(data_feature, axis=0)[i]))
    print("var:"+str(np.var(data_feature, axis=0)[i]))
    print("")
print(minmax)


data_col_0
min:1.0
max:99.0
mean:43.22015
var:140.24748397747882

data_col_1
min:0.0
max:5.0
mean:2.3586
var:1.2923060399996893

data_col_2
min:0.0
max:2.0
mean:1.1708
var:0.31852736000004944

data_col_3
min:13769.0
max:1097453.0
mean:184706.98085
var:10226020824.74352

data_col_4
min:17.0
max:90.0
mean:39.4344
var:165.8094966400004

data_col_5
min:1.0
max:16.0
mean:10.30435
var:6.026421077499933

[[1.000000e+00 9.900000e+01]
 [0.000000e+00 5.000000e+00]
 [0.000000e+00 2.000000e+00]
 [1.376900e+04 1.097453e+06]
 [1.700000e+01 9.000000e+01]
 [1.000000e+00 1.600000e+01]]


# Normalization

In [193]:
# 거리를 구할 때 유클리드 거리 (l2 norm)으로 구할 예정
# 한 요소가 너무 커버리면 그 요소에 다른게 trivial 해진다.
# normalization을 해주자.
normalized = np.empty((20000,6))
#print(normalized.shape[1])
for i in range(normalized.shape[1]):
    normalized[:, i] = (data_feature[:, i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])
print(normalized)


[[0.39795918 0.4        0.5        0.06745786 0.10958904 0.53333333]
 [0.39795918 0.4        0.5        0.22659188 0.50684932 0.6       ]
 [0.39795918 0.4        0.5        0.22427571 0.35616438 0.86666667]
 ...
 [0.39795918 0.4        0.5        0.15553796 0.20547945 0.26666667]
 [0.39795918 0.6        0.5        0.03829806 0.4109589  0.6       ]
 [0.5        0.8        0.5        0.04785066 0.24657534 0.8       ]]
[0.39795918 0.4        0.5        0.06745786 0.10958904 0.53333333]


# augment feature and class data

In [195]:
data_augmented = np.concatenate((normalized, data_class.reshape(20000, 1)), axis = 1)
#print(data_augmented)
#print(data_augmented)
#print(data_augmented[0][6].dtype)

#unsatisfied 는 0으로, satisfied는 1로 바꾼다.
np.putmask(data_augmented, data_augmented == 'satisfied', 1)
np.putmask(data_augmented, data_augmented == 'unsatisfied', 0)
print(data_augmented[0][0].dtype)

<U18


# KNN
## train data set, test data set separation

In [197]:
#샘플 인덱스들의 배열
#sample_train = np.empty((0,2000), int)

#for i in range(10):
#    sample_train = np.append(sample_train, np.random.choice(20000, 2000, replace=False).reshape(1, 2000), axis=0)
#    print(sample_train[i])
#print(sample_train.shape)


#데이터의 총 개수는 20000개로, 1:9로 나누면 2000, 18000이다
#훈련데이터에 사용할 데이터를 18000, 테스트에 사용할 데이터를 2000개로 설정하자.
#0~19999의 숫자 중에서 2000개를 샘플링한다.


#무작위로 배열하고 2000:18000으로 split하는게 나을 듯..
#print(data_feature.shape)
# sample_train_x = np.empty((10, 18000, 6), np.uint64)
# sample_train_y = np.empty((10, 18000, 1), np.uint64)
# sample_test_x = np.empty((10, 2000, 6), np.uint64)
# sample_test_y = np.empty((10, 2000, 1), np.uint64)
#data_feature_mixed = np.random.permutation(data_feature)
#print(data_feature_mixed)

sample_train_x = np.empty((10, 18000, 6))
sample_train_y = np.empty((10, 18000, 1))
sample_test_x = np.empty((10, 2000, 6))
sample_test_y = np.empty((10, 2000, 1))

for i in range(10):
    data_augmented_mixed = np.random.permutation(data_augmented).reshape(1, 20000, 7)
    #print(data_augmented_mixed[0][0][6])
    sample_train_x[i], sample_test_x[i] = data_augmented_mixed[0, 2000:, :6], data_augmented_mixed[0, :2000, :6]
    sample_train_y[i], sample_test_y[i] = data_augmented_mixed[0, 2000:, 6].reshape(1, 18000, 1), data_augmented_mixed[0, :2000, 6].reshape(1, 2000, 1)
#print(sample_train.shape)
#print(sample_test.shape)
#print(sample_train)
print(sample_train_x[0])
#print(sample_test_x[0])
print(sample_train_y[0])
#print(sample_test_y[0])

[[0.5        0.2        0.5        0.00866304 0.35616438 0.86666667]
 [0.44897959 0.4        0.5        0.09813931 0.43835616 1.        ]
 [0.39795918 0.4        1.         0.09129691 0.09589041 0.66666667]
 ...
 [0.39795918 0.4        0.5        0.27966824 0.08219178 0.6       ]
 [0.39795918 0.4        0.5        0.19688304 0.2739726  0.13333333]
 [0.19387755 0.4        1.         0.30357835 0.         0.4       ]]
[[1.]
 [1.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


## compute distance > 2for loops

In [200]:
# 입력으로 들어오는 feature에 대해서 각각의 요소에 대해 거리를 구한다. 
# feature에 들어있는 모든 요소는 정수형이기 때문에 거리를 구할 수 있다.

number_train = sample_train_x.shape[1]
number_test = sample_test_x.shape[1]
euclid_dists = np.zeros((number_test, number_train))
#print(number_train, number_test)
#print(sample_train_x[0][0])
#print(sample_test_x[0][0])
#print(np.square(sample_train_x[0][0] - sample_test_x[0][0]))
#print(np.sum(np.square(sample_train_x[0][0] - sample_test_x[0][0])))
#print(np.sqrt(np.sum(np.square(sample_train_x[0][0] - sample_test_x[0][0]))))

# RuntimeWarning: invalid value encountered in sqrt
# 루트를 계산하는데 음수가 나오는 경우에 주로 발생
# 설계 상 음수가 나올 수 없음.. 오버플로가 일어남

#print(sample_test_x[0].shape)
#print(sample_train_x[0].shape)

# 바로 l2 norm을 계산하기에는 4번째 요소가 너무 큼.
# >> overflow
# 데이터타입을 np.uint64로 변경
# 또는 normalization을 할 수도?

#TODO: sample 10개 돌리는 걸로 바꾸기
#for문을 2중 루프로 돌리니 너무 느리다.
#본인 컴퓨터 기준 샘플당 6분 6초
#10개의 샘플을 돌리면 1시간.
for i in range(number_test):
    for j in range(number_train):
        euclid_dists[i, j] = np.sqrt(np.sum(np.square(sample_test_x[0][i] - sample_train_x[0][j])))
print(euclid_dists)

[[0.47276054 0.38550258 0.74364463 ... 0.52619022 0.68999522 0.86617528]
 [0.41257385 0.63543202 0.68185983 ... 0.49982231 0.57948379 0.77835969]
 [0.42755047 0.43511719 0.5863274  ... 0.23054328 0.55647562 0.70007705]
 ...
 [0.85131655 0.92115712 0.33082672 ... 0.57697909 0.64845982 0.14591118]
 [0.80928346 0.77013434 0.31551814 ... 0.59877582 0.86365039 0.40608673]
 [0.66833082 0.7388685  0.10024286 ... 0.56447096 0.73257374 0.41246981]]


In [209]:
# 테스트용 칸
#TODO: 그래서 가장 가까운거 찾았어. 그럼 그 인덱스는 어떻게 알까나?
#TODO: 그건 개쩌는 np.argsort를 쓰면 됨
print(euclid_dists.shape)
print(euclid_dists[0])
print(euclid_dists_matrix.shape)

(2000, 18000)
[0.47276054 0.38550258 0.74364463 ... 0.52619022 0.68999522 0.86617528]
(2000, 18000)


## compute distance > matrix

In [202]:
euclid_dists_matrix = np.zeros((number_test, number_train), dtype=np.uint64)

test_matrix = np.sum(np.square(sample_test_x[0]), axis=1).reshape(number_test, 1)
train_matrix = np.sum(np.square(sample_train_x[0]), axis=1).reshape(1, number_train)
euclid_dists_matrix = np.sqrt(test_matrix + train_matrix -2*np.dot(sample_test_x[0], sample_train_x[0].T))

print(euclid_dists_matrix)

#이렇게 계산하면 0.9초만에 된다.
#10번하면 9초!

[[0.47276054 0.38550258 0.74364463 ... 0.52619022 0.68999522 0.86617528]
 [0.41257385 0.63543202 0.68185983 ... 0.49982231 0.57948379 0.77835969]
 [0.42755047 0.43511719 0.5863274  ... 0.23054328 0.55647562 0.70007705]
 ...
 [0.85131655 0.92115712 0.33082672 ... 0.57697909 0.64845982 0.14591118]
 [0.80928346 0.77013434 0.31551814 ... 0.59877582 0.86365039 0.40608673]
 [0.66833082 0.7388685  0.10024286 ... 0.56447096 0.73257374 0.41246981]]


## difference between two calculated distance matrix

In [205]:
print(np.sum(euclid_dists - euclid_dists_matrix))

-4.1285255701159255e-07


## compute prediction

In [226]:
prediction = np.empty(number_test)

# 과제 참고자료에서 5라고 나옴.
# 우선 5로 해보자.
k = 5

for i in range(number_test):
    neareast_class = []
    #print(sample_train_y.shape)
    #print(np.argsort(euclid_dists_matrix[i]))
    #print(np.argsort(euclid_dists_matrix[i]).shape)
    neareast_class = (sample_train_y[0][np.argsort(euclid_dists_matrix[i])])[:k]
    #print(neareast_class)
    #print(neareast_class.shape)
    #print(neareast_class[0].dtype)
    neareast_class = neareast_class.astype(int)
    prediction[i] = np.argmax(np.bincount(neareast_class.reshape(5,)))
print(prediction)

[1. 0. 0. ... 0. 0. 0.]


# compare prediction and real class

In [241]:
#print(sample_test_y[0].shape) 
# (2000, 1)
#print(prediction.reshape(number_test, 1).shape)
# (2000, 1)

print(sample_test_y[0].reshape(2000, ) - prediction)
isAnswer = sample_test_y[0].reshape(2000, ) - prediction
print(len(isAnswer[isAnswer == 0]))

number_correct = len(isAnswer[isAnswer == 0])

print("accuracy: "+str((number_correct/number_test)*100))

#TODO: accuracy를 올리려면? 해밍 거리밖에 없나

[0. 0. 1. ... 0. 0. 0.]
1495
accuracy: 74.75
